### 인구소멸 위기지역 - 전처리 (Part 1)

In [1]:
import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('data/시군구_성_연령_5세_별_인구2_2020.csv', encoding='euc-kr')
df
print(df.shape)
df.head(260).tail(5)

In [6]:
sum(df.행정구.notnull())

129

In [7]:
# 기초자치단체(시군구)에 행정구가 없으면 
df['시군구'][df.행정구.notnull()] = df.행정구[df.행정구.notnull()]
df.head(260).tail()

,광역시도,시군구,행정구,구분,인구수,0 - 4세,5 - 9세,10 - 14세,15 - 19세,20 - 24세,...,65 - 69세,70 - 74세,75 - 79세,80세 이상,80 - 84세,85세 이상,85 - 89세,90 - 94세,95 - 99세,100세 이상
255,경기도,소계,소계,계,1181470,43910,56741,57093,63808,86365,...,43315,31267,23774,27796,16028,11768,8044,2957,671,96
256,경기도,소계,소계,남자,594380,22626,29290,29411,33178,44785,...,20789,14491,10013,9142,5900,3242,2414,678,137,14
257,경기도,소계,소계,여자,587090,21284,27451,27682,30630,41581,...,22527,16776,13761,18654,10129,8526,5630,2280,535,82
258,경기도,장안구,장안구,계,273908,7659,11548,13483,16100,22511,...,11311,8549,6870,8443,4806,3637,2459,944,207,28
259,경기도,장안구,장안구,남자,137852,3904,6034,6954,8498,12166,...,5418,3905,2892,2846,1819,1027,755,225,46,2


In [ ]:
# 소계 데이터 지우기
df = df[df['시군구'] != '소계']
print(df.shape)
df.head()

In [11]:
# 20~39세, 65세 이상 컬럼 만들기
df['20-39세'] = df['20 - 24세'] + df['25 - 29세'] + df['30 - 34세'] + df['35 - 39세']
df['65세이상'] = df['65 - 69세'] + df['70 - 74세'] + df['75 - 79세'] + df['80세 이상']
df = df[['광역시도','시군구','구분','인구수','20-39세','65세이상']]
df.head()

,광역시도,시군구,구분,인구수,20-39세,65세이상
6,서울특별시,종로구,계,144866,43061,25597
7,서울특별시,종로구,남자,70613,21327,11361
8,서울특별시,종로구,여자,74253,21736,14236
9,서울특별시,중구,계,121520,38455,21878
10,서울특별시,중구,남자,59536,18864,9600


In [13]:
# pivot_table을 만들어서 시군구 단위로 그룹핑
pop = pd.pivot_table(df,
                index=['광역시도'],
                columns=['구분'],
                values=['인구수','20-39세','65세이상'])
print(pop.shape)
pop.head()

(17, 9)


20-39세                                     65세이상                \
구분                계            남자            여자             계            남자   
광역시도                                                                          
강원도    19275.444444  10474.222222   8802.166667  17074.611111   7354.500000   
경기도    87726.119048  45619.190476  42107.952381  40225.666667  17682.642857   
경상남도   35942.681818  19322.500000  16621.136364  25451.636364  10690.909091   
경상북도   24384.416667  13363.125000  11022.291667  23227.250000   9765.083333   
광주광역시  79218.400000  40841.400000  38378.200000  39737.600000  16925.800000   

                               인구수                                
구분               여자              계             남자             여자  
광역시도                                                              
강원도     9721.111111   84927.388889   42749.277778   42178.500000  
경기도    22543.880952  314767.071429  158300.357143  156467.023810  
경상남도   14762.181818  151456.363636   76216.363636   75240.227273  
경상북도   13463.250000  109823.916667   55303.041667   54521.125000  
광주광역시  22812.600000  288893.600000  142901.400000  145992.400000

In [14]:
# 인구 소멸비율 계산
pop['소멸비율'] = pop['20-39세','여자'] / pop['65세이상','계']
pop.head(3)

20-39세                                     65세이상                \
구분               계            남자            여자             계            남자   
광역시도                                                                         
강원도   19275.444444  10474.222222   8802.166667  17074.611111   7354.500000   
경기도   87726.119048  45619.190476  42107.952381  40225.666667  17682.642857   
경상남도  35942.681818  19322.500000  16621.136364  25451.636364  10690.909091   

                              인구수                                    소멸비율  
구분              여자              계             남자             여자            
광역시도                                                                       
강원도    9721.111111   84927.388889   42749.277778   42178.500000  0.515512  
경기도   22543.880952  314767.071429  158300.357143  156467.023810  1.046793  
경상남도  14762.181818  151456.363636   76216.363636   75240.227273  0.653048

In [15]:
pop['소멸위기지역'] = pop.소멸비율 < 0.5
pop['소멸위기고위험지역'] = pop.소멸비율 < 0.2
pop.head(3)

20-39세                                     65세이상                \
구분               계            남자            여자             계            남자   
광역시도                                                                         
강원도   19275.444444  10474.222222   8802.166667  17074.611111   7354.500000   
경기도   87726.119048  45619.190476  42107.952381  40225.666667  17682.642857   
경상남도  35942.681818  19322.500000  16621.136364  25451.636364  10690.909091   

                              인구수                                    소멸비율  \
구분              여자              계             남자             여자             
광역시도                                                                        
강원도    9721.111111   84927.388889   42749.277778   42178.500000  0.515512   
경기도   22543.880952  314767.071429  158300.357143  156467.023810  1.046793   
경상남도  14762.181818  151456.363636   76216.363636   75240.227273  0.653048   

     소멸위기지역 소멸위기고위험지역  
구분                     
광역시도                   
강원도   False     False  
경기도   False     False  
경상남도  False     False

In [16]:
# 인구소멸 위기지역
crisis_region = pop[pop.소멸위기지역].index.get_level_values(1)
print(len(crisis_region))
print(high_crisis_region)

IndexError: Too many levels: Index has only 1 level, not 2

- 컬럼명 변경(이중 인덱스 --> 단일 인덱스)

In [17]:
pop.columns.get_level_values(1)

Index(['계', '남자', '여자', '계', '남자', '여자', '계', '남자', '여자', '', '', ''], dtype='object', name='구분')

In [18]:
for i in len(pop.columns.get_level_values(0)):
    print(pop.columns.get_level_values(0)[i] + pop.columns.get_level_values(1)[i])

TypeError: 'int' object is not iterable

In [ ]:
new_columns = []
for i in range(len(pop))